In [1]:
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from skimage.transform import resize

# Prepare Data

In [2]:
train=pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
train.head()

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity
0,0007de18844b0dbbb5e1f607da0606e0,0,1,1,1,0,0,1,0,0,0,0,0,63
1,0009c66b9439883ba2750fb825e1d7db,0,1,1,0,0,0,0,0,0,0,0,0,42
2,0013fd999caf9a3efe1352ca1b0d937e,0,1,1,1,0,0,0,0,1,1,0,0,28
3,0018df346ac9c1d8413cfcc888ca8246,0,1,1,1,0,0,0,0,0,0,0,0,15
4,001dc955e10590d3ca4673f034feeef2,0,0,0,1,0,0,1,0,0,0,0,0,72


from train datafram, we might use Id in for loop to extract image

## Extract and resize image

I decided to resize all image into 128 x 128 pixel

In [3]:
all_image=[]
count=0
for index in range(train.shape[0]):
    id=train.loc[index,'Id']
    path='../input/petfinder-pawpularity-score/train/'+str(id)+'.jpg'
    im_array=cv2.imread(path)
    im_array=resize(im_array,(128,128),anti_aliasing=True)
    all_image.append(im_array)
    
    count+=1
    progress=(count/len(train['Id']))*100
    print('progress =',round(progress,2), '%',end='\r')

In [4]:
X=np.array(all_image)
X=X.reshape(-1,128,128,3)

del all_image
X.shape

(9912, 128, 128, 3)

In [5]:
y=train['Pawpularity']
y.shape

(9912,)

# Neural Network

I adapt neural network structure from this [notebook](https://www.kaggle.com/guidosalimbeni/regression-with-convolutional-neural-network-keras#Build-and-train-the-CNN-model)

In [6]:
model=tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(64,(3,3),activation='relu',input_shape=(128,128,3),padding='same'))
model.add(tf.keras.layers.MaxPool2D((2,2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(512,activation='relu'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(256,activation='relu'))
model.add(tf.keras.layers.Dense(128,activation='relu'))
model.add(tf.keras.layers.Dense(1,activation='linear'))
model.compile(loss='mean_squared_error',optimizer='adam')
model.summary()

2022-01-03 15:01:04.271238: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-03 15:01:04.380237: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-03 15:01:04.381036: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-03 15:01:04.382256: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 262144)            0         
_________________________________________________________________
dense (Dense)                (None, 512)               134218240 
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 128)               3

In [7]:
model.fit(X,y,epochs=50)

2022-01-03 15:01:11.073250: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1948778496 exceeds 10% of free system memory.
2022-01-03 15:01:13.546246: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1948778496 exceeds 10% of free system memory.
2022-01-03 15:01:14.994981: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/50


2022-01-03 15:01:16.248106: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


310/310 [==============================] - 14s 23ms/step - loss: 483.9475
Epoch 2/50
310/310 [==============================] - 7s 23ms/step - loss: 447.3421
Epoch 3/50
310/310 [==============================] - 7s 23ms/step - loss: 439.9470
Epoch 4/50
310/310 [==============================] - 7s 23ms/step - loss: 420.5487
Epoch 5/50
310/310 [==============================] - 7s 23ms/step - loss: 366.7389
Epoch 6/50
310/310 [==============================] - 7s 23ms/step - loss: 301.4953
Epoch 7/50
310/310 [==============================] - 7s 23ms/step - loss: 222.5154
Epoch 8/50
310/310 [==============================] - 7s 24ms/step - loss: 148.6778
Epoch 9/50
310/310 [==============================] - 7s 23ms/step - loss: 94.6122
Epoch 10/50
310/310 [==============================] - 7s 23ms/step - loss: 65.7006
Epoch 11/50
310/310 [==============================] - 7s 23ms/step - loss: 44.7244
Epoch 12/50
310/310 [==============================] - 7s 23ms/step - loss: 34.8492
Epo

# Predict pet pawpularity

In [8]:
test=pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
test.head()

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur
0,4128bae22183829d2b5fea10effdb0c3,1,0,1,0,0,1,1,0,0,1,0,1
1,43a2262d7738e3d420d453815151079e,0,1,0,0,0,0,1,1,0,0,0,0
2,4e429cead1848a298432a0acad014c9d,0,0,0,1,0,1,1,1,0,1,1,1
3,80bc3ccafcc51b66303c2c263aa38486,1,0,1,0,0,0,0,0,0,0,1,0
4,8f49844c382931444e68dffbe20228f4,1,1,1,0,1,1,0,1,0,1,1,0


In [9]:
all_image=[]
for index in range(test.shape[0]):
    id=test.loc[index,'Id']
    path='../input/petfinder-pawpularity-score/test/'+str(id)+'.jpg'
    im_array=cv2.imread(path)
    im_array=resize(im_array,(128,128),anti_aliasing=True)
    all_image.append(im_array)

X_test=np.array(all_image).reshape(-1,128,128,3)
del all_image

y_predict=model.predict(X_test)
y_predict=y_predict.reshape(-1,)

submission_df=pd.DataFrame({'Id':test['Id'],'Pawpularity':y_predict})
submission_df

,Id,Pawpularity
0,4128bae22183829d2b5fea10effdb0c3,54.198887
1,43a2262d7738e3d420d453815151079e,55.863613
2,4e429cead1848a298432a0acad014c9d,62.345009
3,80bc3ccafcc51b66303c2c263aa38486,51.611015
4,8f49844c382931444e68dffbe20228f4,46.547592
5,b03f7041962238a7c9d6537e22f9b017,83.747734
6,c978013571258ed6d4637f6e8cc9d6a3,47.924076
7,e0de453c1bffc20c22b072b34b54e50f,62.292130


In [10]:
submission_df.to_csv("submission.csv", index=False)